In [4]:
'''
Imports
'''
import os
import time
import random
import h5py
import csv
from io import BytesIO
import numpy as np
import pandas as pd
import pandas.api.types
from tqdm import tqdm
import torch, torchvision
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import v2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
# import cv2
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
from torch.optim import lr_scheduler
from sklearn.metrics import roc_curve, auc
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
# from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
import lightgbm as lgb

In [22]:
'''
Constants
'''
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
CLASSES = 1
EPOCHS = 15
MIN_EPOCH_TRAIN = 5
PATIENCE = 5
EPSILON = 0.0005
NEG_POS_RATIO = 20
FOLDS = 5
TRAIN_IMAGE = True
TRAIN_METADATA = True

In [ ]:
# Kaggle
TRAIN_HDF5_PATH = "/kaggle/input/isic-2024-challenge/train-image.hdf5"
TEST_HDF5_PATH = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
TRAIN_META = "/kaggle/input/isic-2024-challenge/train-metadata.csv"
TEST_META = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
MODEL_SAVE_PATH_ = "/kaggle/working/"
LOG_FILE_1 = "/kaggle/working/"
LOG_FILE_2 = "/kaggle/working/log_folds.csv"
RESNET34_HAM10000_WEIGHTS_PYTORCH = "/kaggle/input/resnet34_ham10000_11binary_pretrain/pytorch/july12_24/1/ResNet34_HAM10000_1-1.pth"        # change properly
SUBMISSION_FILE_PATH = "/kaggle/working/submission.csv"
METRICS_PLOT_SAVE_PATH = "/kaggle/working/metrics.png"

## Local_Srijan
# TRAIN_HDF5_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Data\\train-image.hdf5"
# TEST_HDF5_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Data\\test-image.hdf5"
# TRAIN_META = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Data\\train-metadata.csv"
# TEST_META = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Data\\test-metadata.csv"
# MODEL_SAVE_PATH_ = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\"
# LOG_FILE_1 = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Codebase\\Classification_v2\\"
# LOG_FILE_2 = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Codebase\\Classification_v2\\log_folds.csv"
# RESNET34_HAM10000_WEIGHTS_PYTORCH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\resnet34-b627a593.pth"        # change properly
# SUBMISSION_FILE_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Codebase\\Classification_v2\\submission.csv"
# METRICS_PLOT_SAVE_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Codebase\\Classification_v2\\metrics.png" 


## Image Data Prep

In [ ]:
'''
Transformations for Image Data
'''
data_transforms_album = {
    "train": A.Compose([
        A.Resize(224, 224),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Downscale(p=0.25),
        A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
        A.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.),
    
    "test": A.Compose([
        A.Resize(224, 224),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

In [ ]:
'''
DataClass for Image Data
'''
class ISIC2024_HDF5_ALBUM(Dataset):
    '''
    With augmentations using albumentations 
    '''
    def __init__(self, hdf5_path, annotations_df=None, transform=None):
        self.hdf5_path = hdf5_path
        self.annotations_df = annotations_df
        self.transform = transform
        self.image_ids = []
        
        self.hdf5_file = h5py.File(self.hdf5_path, 'r')

        if self.annotations_df is not None:
            self.image_ids = annotations_df['isic_id']
            self.labels = annotations_df.set_index('isic_id')['target'].to_dict()
        else:
            self.image_ids = list(self.hdf5_file.keys())
            

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image = np.array(Image.open(BytesIO(self.hdf5_file[image_id][()])))
        
        # image = self.load_image(self.hdf5_file[image_id][()])

        if self.transform:
            # image = self.transform(image)
            image = self.transform(image=image)["image"]        # Albumentations returns a dictionary with keys like 'image', 'mask', etc., depending on the transformations applied.

        # Check for NaN in image
        if torch.isnan(image).any():
            print(f"NaN detected in image {image_id}")

        if self.annotations_df is not None:
            label = self.labels[image_id]
            # Check for NaN in label
            if np.isnan(label):
                print(f"NaN detected in label for image {image_id}")
            return image, label, image_id
        else:
            return image, image_id
        
    # def load_image(self, image_data):
    #     # Decode the image data from HDF5 file using OpenCV
    #     image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
    #     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    #     # image = np.transpose(image, (1, 2, 0))  # Convert HxWxC to CxHxW
    #     return image
    
    def close(self):
        self.hdf5_file.close()

In [ ]:
'''
DataLoader for Image Data
'''
def get_loader(test_hdf5_path, 
               train_labels_df = None, 
               train_hdf5_path = None, 
               dataset_cls=ISIC2024_HDF5_ALBUM,
               train_img_trans=data_transforms_album["train"], 
               test_img_trans=data_transforms_album["test"], 
               n_splits = FOLDS,
               batch=32, 
               seed=42):
    if train_labels_df is not None and train_hdf5_path is not None:
        train_dataset_all = dataset_cls(hdf5_path=train_hdf5_path, annotations_df=train_labels_df, transform=train_img_trans)
        test_dataset = dataset_cls(hdf5_path=test_hdf5_path, transform=test_img_trans)

        train_annotations_all = train_labels_df
        labels = train_annotations_all['target']
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
        train_loaders, val_loaders = [], []
        for train_idx, val_idx in skf.split(train_annotations_all, labels):
            train_subset = Subset(train_dataset_all, train_idx)
            val_subset = Subset(train_dataset_all, val_idx)
            
            train_loaders.append(DataLoader(train_subset, batch_size=batch, shuffle=True))
            val_loaders.append(DataLoader(val_subset, batch_size=batch, shuffle=True))
        
        test_loader = DataLoader(test_dataset, shuffle=False)
        
        return train_loaders, val_loaders, test_loader

    else:
        test_dataset = dataset_cls(hdf5_path=test_hdf5_path, transform=test_img_trans)
        test_loader = DataLoader(test_dataset, shuffle=False)
    
        return test_loader

## MetaData Prep

In [ ]:
'''
Feature Engineering
'''
def feature_engineering(df):
    '''
    
    '''
    df = df.copy()
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
    df["hue_contrast"] = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df["3d_position_distance"] = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    df["combined_anatomical_site"] = df["anatom_site_general"] + "_" + df["tbp_lv_location"]
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"] = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"] = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4

    new_num_cols = [
        "lesion_size_ratio", "lesion_shape_index", "hue_contrast",
        "luminance_contrast", "lesion_color_difference", "border_complexity",
        "color_uniformity", "3d_position_distance", "perimeter_to_area_ratio",
        "lesion_visibility_score", "symmetry_border_consistency", "color_consistency",

        "size_age_interaction", "hue_color_std_interaction", "lesion_severity_index", 
        "shape_complexity_index", "color_contrast_index", "log_lesion_area",
        "normalized_lesion_size", "mean_hue_difference", "std_dev_contrast",
        "color_shape_composite_index", "3d_lesion_orientation", "overall_color_difference",
        "symmetry_perimeter_interaction", "comprehensive_lesion_index",
    ]
    new_cat_cols = ["combined_anatomical_site"]
    return df, new_num_cols, new_cat_cols

In [ ]:
'''
Splits
'''
def get_stratified_splits(metadata_df, n_splits=FOLDS, random_state=42):
    """
    Performs a stratified split of the provided tabular metadata dataframe.
    
    Args:
        metadata_df (pandas.DataFrame): The tabular metadata dataframe to be split.
        n_splits (int, optional): The number of folds to create. Defaults to 5.
        random_state (int, optional): The random state for reproducibility. Defaults to 42.
        
    Returns:
        list: A list of tuples, where each tuple contains the train and validation dataframes for a fold.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    splits = []
    for train_idx, val_idx in skf.split(metadata_df, metadata_df['target']):
        train_df = metadata_df.iloc[train_idx].reset_index(drop=True)
        val_df = metadata_df.iloc[val_idx].reset_index(drop=True)
        splits.append((train_df, val_df))
    
    return splits

In [ ]:
'''
Model
'''

__n_estimators         = random.choice([1400, 1500, 2000])                  # 800,900,1000,1100,1200,1300,
__learning_rate        = random.choice([0.003, 0.002, 0.001])               # 0.005,0.004,
__lambda_l1            = random.choice([0.14, 0.21, 0.27, 0.37])
__lambda_l2            = random.choice([0.7, 1.0, 1.47, 1.77, 2.77])
__pos_bagging_fraction = random.choice([0.74, 0.75, 0.77, 0.777])
__neg_bagging_fraction = random.choice([0.04, 0.05, 0.07, 0.077])
__feature_fraction     = random.choice([0.5, 0.54, 0.57, 0.7, 0.77, 0.777])
__num_leaves           = random.choice([16, 20, 24, 30, 33, 37])            # 24,30,31,32,33,37
__min_data_in_leaf     = random.choice([16, 20, 24, 40, 50, 57])            # 40,45,50,55,57

lgb_params = {
    'objective': 'binary',
    "random_state": 42,
    "n_estimators":__n_estimators,
    'learning_rate':__learning_rate,
    'num_leaves':__num_leaves,
    'min_data_in_leaf':__min_data_in_leaf,
    'bagging_freq': 1,
    'pos_bagging_fraction':__pos_bagging_fraction,
    'neg_bagging_fraction':__neg_bagging_fraction,
    'feature_fraction':__feature_fraction,
    'lambda_l1':__lambda_l1,
    'lambda_l2':__lambda_l2,
    "verbosity": -1,
    # "extra_trees": True
}

## Helper Functions

In [23]:
def train_image(epochs, model, learning_rate, train_dl, val_dl, min_epoch_train, patience, epsilon, log_file, model_save_path, criterion = nn.BCEWithLogitsLoss()):
    '''
    Training function for ISIC-2024 competition data
    Parameters:
        epochs: Number of epoches
        model: Model 
        learning_rate: model learning rate
        train_dl: Training data loader
        val_dl: Validation data loader
        min_epoch_train: Train for minimum epoches after which early-stopping kicks in
        patience: Patience for early-stopping
        epsilon: minimum required improvement in order to go on beyond early-stopping
        log_file: log file location to save logs
        model_save_path: location for saving trained model 
        criterion: Loss function, defaults to BCE with logit loss function
    '''
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)  # Initialize CosineAnnealingLR scheduler
    # scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10,  15], gamma=0.1)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    scaler = torch.amp.GradScaler('cuda')

    best_val_pauc = -1.0  # Initialize with a very low value
    current_patience = 0  # Initialize patience counter

    with open(log_file, "w", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(['Epoch', 'Learning Rate', 'Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy', 'Validation Precision', 'Validation Recall', 'Validation F1 Score', 'Validation pAUC'])
        
        for epoch in range(epochs):
            print(f"\n | Epoch: {epoch+1}")
            total_loss = 0
            num_corr = 0
            num_samp = 0
            loop = tqdm(train_dl)
            model.train()

            for batch_idx, (inputs, labels, _) in enumerate(loop):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                with torch.amp.autocast('cuda'):
                    outputs = model(inputs).squeeze(1)
                    loss = criterion(outputs, labels.float())
                
                if torch.isnan(loss):
                    print(f"NaN loss detected at batch {batch_idx}")
                    continue
                
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                
                preds = torch.sigmoid(outputs)
                num_corr += ((preds > 0.5) == labels).sum()
                num_samp += preds.size(0)
                total_loss += loss.item()
                loop.set_postfix(loss=loss.item())
            
            avg_loss = total_loss / len(train_dl)
            acc = num_corr / num_samp
            print(f"| Epoch {epoch+1}/{epochs} total training loss: {total_loss}, average training loss: {avg_loss}.")
            
            print("On Validation Data:")
            
            model.eval()
            with torch.inference_mode():
                val_loss, val_acc, val_pre, val_rec, val_f1, val_pauc = evaluate(val_dl, model, criterion)
            print("learning rate:", scheduler.get_last_lr()[0])
            row = [epoch+1, scheduler.get_last_lr()[0], avg_loss, acc.item(), val_loss, val_acc, val_pre, val_rec, val_f1, val_pauc]
            csv_writer.writerow(row)

            if epoch + 1 > min_epoch_train:
                '''
                early-stopping code
                '''
                if val_pauc > best_val_pauc and (val_pauc - best_val_pauc) > epsilon:
                    best_val_pauc = val_pauc
                    print(f'Validation pAUC improved by more than {epsilon}, ({best_val_pauc} > {best_val_pauc})); saving model...')
                    checkpoint = {
                        "state_dict": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                    }
                    save_checkpoint(checkpoint, model_save_path)
                    print(f'Model saved at {model_save_path}')
                    current_patience = 0  # Reset patience if there's an improvement
                else:
                    current_patience += 1
                    print(f'Validation pAUC did not improve. Patience left: {patience - current_patience}')
                    
                    if current_patience >= patience:
                        print(f'\n---Early stopping at epoch {epoch+1}.---')
                        break
            else:
                '''
                train for at least min_epoch_train epochs and keep saving best
                '''
                if val_pauc > best_val_pauc:
                    best_val_pauc = val_pauc
                    checkpoint = {
                        "state_dict": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                    }
                    save_checkpoint(checkpoint, model_save_path)
                    print(f'Model saved at {model_save_path}')

            print(f'Current Best Validation pAUC: {best_val_pauc}')

            scheduler.step()  # Update learning rate for next epoch
        
    print('Training complete.')

    return best_val_pauc


def pauc_above_tpr(y_true, y_pred, min_tpr=0.80):
    '''
    Custom metric according to competition
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve
    https://www.kaggle.com/code/metric/isic-pauc-abovetpr
    '''
    y_true = abs(np.array(y_true) - 1)
    y_pred = -1.0 * np.array(y_pred)
    
    # Check for NaN values
    if np.isnan(y_true).any() or np.isnan(y_pred).any():
        print("NaN values detected in inputs to pauc_above_tpr")
        return 0
    
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    max_fpr = 1 - min_tpr

    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    
    if len(fpr) < 2:
        print("Warning: Not enough points to compute pAUC. Returning 0.")
        return 0
    
    partial_auc = auc(fpr, tpr)

    return partial_auc


def evaluate(loader, model, criterion):
    '''
    Evaluate function for validation set
    '''
    metric = BinaryF1Score(threshold=0.5).to(DEVICE)
    prec = BinaryPrecision(threshold=0.5).to(DEVICE)
    recall = BinaryRecall(threshold=0.5).to(DEVICE)
    acc = BinaryAccuracy(threshold=0.5).to(DEVICE)
    loss = 0.0
    num_corr = 0
    num_samp = 0
    all_preds = []
    all_labels = []
    
    model.eval()
    with torch.no_grad():
        for inputs, labels, _ in tqdm(loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs).squeeze(1)
            
            # Check for NaN in outputs
            if torch.isnan(outputs).any():
                print("NaN detected in model outputs")
                continue
            
            loss += criterion(outputs, labels.float()).item()
            preds = torch.sigmoid(outputs)
            num_corr += ((preds > 0.5) == labels).sum()
            num_samp += preds.size(0)
            metric.update(preds, labels)
            prec.update(preds, labels)
            recall.update(preds, labels)
            acc.update(preds, labels)
            all_preds.extend(preds.cpu().detach().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = loss / len(loader)
    accu = float(num_corr) / float(num_samp)
    pauc = pauc_above_tpr(all_labels, all_preds)
    
    print(f"Total loss: {loss}, Average loss: {avg_loss}")
    print(f"Got {num_corr}/{num_samp} correct with accuracy {accu*100:.2f}")
    print(f"pAUC above 80% TPR: {pauc:.3f}, Accuracy: {acc.compute().item():.3f}, precision: {prec.compute().item():.3f}, recall: {recall.compute().item():.3f}, F1Score: {metric.compute().item():.3f}")
    model.train()

    return avg_loss, acc.compute().item(), prec.compute().item(), recall.compute().item(), metric.compute().item(), pauc


def save_checkpoint(state, filename="my_checkpoint.pth"):
    '''
    To save model while training 
    Saves in working directory by default
    '''
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model):
    '''
    
    '''
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def load_model(model_save_path = None, imagenet_weights_path = None):   # Use this for submission rather than load_checkpoint() defined above
    '''
    To load model during evaluation on test set
    '''
    if model_save_path:
        model = torchvision.models.resnet34(weights=None)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(in_features=num_ftrs, out_features=1)
        model.load_state_dict(torch.load(model_save_path)["state_dict"])
        model.to(DEVICE)
        model.eval()
    else:
        model = torchvision.models.resnet34(weights=None)
        model.load_state_dict(torch.load(imagenet_weights_path))
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(in_features=num_ftrs, out_features=1)
        model.to(DEVICE)
        
    return model


def create_submission(submission_file_path, train_image, train_metadata, image_model = None, test_loader = None, meta_models = None, test_meta = None, train_cols = None, cat_cols = None):
    '''
    To predict class probabilities on test data and generate submission.csv file
    '''
    predictions = []
    image_ids = []
    if train_image and not train_metadata:
        with torch.no_grad():
            for inputs, image_names in tqdm(test_loader, desc="Evaluating"):
                inputs = inputs.to(DEVICE)
                outputs = image_model(inputs).squeeze(1)
                probs = torch.sigmoid(outputs)
                predictions.extend(probs.cpu().numpy())
                image_ids.extend(image_names)  # Append all image names from the batch

    elif train_metadata and not train_image:
        df_test = pd.read_csv(test_meta, low_memory=False)
        image_ids = list(df_test['isic_id'])
        df_test, _, _ = feature_engineering(df_test.copy())
        category_encoder = OrdinalEncoder(
            categories='auto',
            dtype=int,
            handle_unknown='use_encoded_value',
            unknown_value=-2,
            encoded_missing_value=-1,
        )
        X_cat = category_encoder.fit_transform(df_test[cat_cols])
        for c, cat_col in enumerate(cat_cols):
            df_test[cat_col] = X_cat[:, c]
        predicted_probabilities_list = []
        for model in meta_models:
            raw_preds = model.predict(df_test[train_cols])
            probs = 1 / (1 + np.exp(-raw_preds))
            predicted_probabilities_list.append(probs)
#         predictions = np.mean([model.predict(df_test[train_cols]) for model in meta_models], 0)
        predictions = np.mean(predicted_probabilities_list, axis=0)
        
    elif train_image and train_metadata:
        predictions_1 = []
        predictions_2 = []
        # Image part
        with torch.no_grad():
            for inputs, image_names in tqdm(test_loader, desc="Evaluating"):
                inputs = inputs.to(DEVICE)
                outputs = image_model(inputs).squeeze(1)
                probs = torch.sigmoid(outputs)
                predictions_1.extend(probs.cpu().numpy())
                image_ids.extend(image_names)  # Append all image names from the batch
        # Meta part
        df_test = pd.read_csv(test_meta, low_memory=False)
        df_test, _, _ = feature_engineering(df_test.copy())
        category_encoder = OrdinalEncoder(
            categories='auto',
            dtype=int,
            handle_unknown='use_encoded_value',
            unknown_value=-2,
            encoded_missing_value=-1,
        )
        X_cat = category_encoder.fit_transform(df_test[cat_cols])
        for c, cat_col in enumerate(cat_cols):
            df_test[cat_col] = X_cat[:, c]
        predicted_probabilities_list = []
        for model in meta_models:
            raw_preds = model.predict(df_test[train_cols])
            probs = 1 / (1 + np.exp(-raw_preds))
            predicted_probabilities_list.append(probs)
#         predictions = np.mean([model.predict(df_test[train_cols]) for model in meta_models], 0)
        predictions_2 = np.mean(predicted_probabilities_list, axis=0)

        predictions = (predictions_1 + predictions_2) / 2
    else:
        print("Bruh")

    # Check if the lengths match
    if len(image_ids) != len(predictions):
        print(f"Warning: Number of image IDs ({len(image_ids)}) does not match number of predictions ({len(predictions)})")

    # Create DataFrame
    submission_df = pd.DataFrame({
        'isic_id': image_ids,
        'target': predictions
    })

    # Save to CSV
    submission_df.to_csv(submission_file_path, index=False)
    print(f"Submission file saved to {submission_file_path}")


def visualize_train_images(images, titles=None):
    '''
    
    '''
    plt.figure(figsize=(15, 5))
    for i, image in enumerate(images):
        plt.subplot(1, len(images), i + 1)
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).numpy()  # Change from CxHxW to HxWxC and convert to numpy
        plt.imshow(image)
        if titles:
            plt.title(f"{titles[0][i]} (label: {titles[1][i]})")
        plt.axis('off')
    plt.show()
    """ Usage
    indices = np.random.choice(len(train_dataset_album), size=3, replace=False)
    images, label, image_ids  = zip(*[train_dataset_album[i] for i in indices])
    visualize_train_images(images, titles=[image_ids, label])
    """


def visualize_train_images(images, titles=None):
    '''
    
    '''
    plt.figure(figsize=(15, 5))
    for i, image in enumerate(images):
        plt.subplot(1, len(images), i + 1)
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).numpy()  # Change from CxHxW to HxWxC and convert to numpy
        plt.imshow(image)
        if titles:
            plt.title(f"{titles[0][i]} (label: {titles[1][i]})")
        plt.axis('off')
    plt.show()

    """ usage
    indices = np.random.choice(len(train_dataset), size=3, replace=False)
    images, label, image_ids  = zip(*[train_dataset[i] for i in indices])
    visualize_train_images(images, titles=[image_ids, label])
    """


def visualize_test_images(images, titles=None):
    '''
    
    '''
    plt.figure(figsize=(15, 5))
    for i, image in enumerate(images):
        plt.subplot(1, len(images), i + 1)
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).numpy()  # Change from CxHxW to HxWxC and convert to numpy
        plt.imshow(image)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()
    ''' Usage
    indices = np.random.choice(len(test_dataset), size=3, replace=False)
    images, image_ids  = zip(*[test_dataset[i] for i in indices])
    visualize_test_images(images, titles=image_ids)    
    '''


def plot_metrics_from_files(file_paths, save_path=None):
    '''
    
    '''
    num_files = len(file_paths)
    rows = 4  # Four rows for four types of plots
    cols = num_files  # One column per file

    fig, axes = plt.subplots(rows, cols, figsize=(10 * cols, 10))  # Create a grid of subplots
    
    if num_files == 1:
        axes = axes[:, None]
    
    for i, file_path in enumerate(file_paths):
        # Read the CSV file
        df = pd.read_csv(file_path)

        # Extract relevant columns
        epochs = df['Epoch']
        learning_rate = df['Learning Rate']
        train_loss = df['Training Loss']
        valid_loss = df['Validation Loss']
        valid_pAUC = df['Validation pAUC']
        valid_acc = df['Validation Accuracy']
        valid_prec = df['Validation Precision']
        valid_recall = df['Validation Recall']
        valid_f1score = df['Validation F1 Score']

        # Plot training loss and validation loss
        axes[0, i].plot(epochs, train_loss, label='Training Loss', marker='o')
        axes[0, i].plot(epochs, valid_loss, label='Validation Loss', marker='o')
        axes[0, i].set_title(f'File: {file_path.split("/")[-1]}')
        axes[0, i].set_xlabel('Epochs')
        axes[0, i].set_ylabel('Loss')
        axes[0, i].legend()

        # Plot epoch vs learning rate
        axes[1, i].plot(epochs, learning_rate, label='Learning Rate', marker='o', color='orange')
        axes[1, i].set_title(f'File: {file_path.split("/")[-1]}')
        axes[1, i].set_xlabel('Epochs')
        axes[1, i].set_ylabel('Learning Rate')
        axes[1, i].legend()

        # Plot validation pAUC
        axes[2, i].plot(epochs, valid_pAUC, label='Validation pAUC', marker='o', color='green')
        axes[2, i].set_title(f'File: {file_path.split("/")[-1]}')
        axes[2, i].set_xlabel('Epochs')
        axes[2, i].set_ylabel('Validation pAUC')
        axes[2, i].legend()

        # Plot accuracy, precision, recall, f1-score
        axes[3, i].plot(epochs, valid_acc, label = "Validation Accuracy", marker = 'o')
        axes[3, i].plot(epochs, valid_prec, label = "Validation Precision", marker = 'o')
        axes[3, i].plot(epochs, valid_recall, label = "Validation Recall", marker = 'o')
        axes[3, i].plot(epochs, valid_f1score, label = "Validation F1 Score", marker = 'o')
        axes[3, i].set_title(f'File: {file_path.split("/")[-1]}')
        axes[3, i].set_xlabel('Epochs')
        axes[3, i].set_ylabel('Validation Metrics')
        axes[3, i].legend()

    plt.tight_layout()  
    
    if save_path:
        plt.savefig(save_path)

    plt.show()


def lesgooo(TRAIN_IMAGE, TRAIN_METADATA):
    '''
    
    '''
    annotations_df_full = pd.read_csv(TRAIN_META, low_memory=False)
    df_positive_all = annotations_df_full[annotations_df_full["target"] == 1].reset_index(drop=True)
    df_negative_all = annotations_df_full[annotations_df_full["target"] == 0].reset_index(drop=True)
    df_negative_trunc = df_negative_all.sample(df_positive_all.shape[0]*NEG_POS_RATIO)
    annotations_df_trunc = pd.concat([df_positive_all, df_negative_trunc]).sample(frac=1).reset_index()

    if TRAIN_IMAGE and not TRAIN_METADATA:
        train_loaders, val_loaders, _ = get_loader(
            test_hdf5_path=TEST_HDF5_PATH,
            train_labels_df=annotations_df_trunc,
            train_hdf5_path=TRAIN_HDF5_PATH
        )
        val_pAUC = []
        with open(LOG_FILE_2, 'w', newline="") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(['Fold', "Model_Name", "avg_val_pAUC"])
            for fold in range(FOLDS):
                model_resnet = load_model(model_save_path=RESNET34_HAM10000_WEIGHTS_PYTORCH)
                print(f"---------------\nTraining for fold: {fold+1}:\n---------------")
                val_pAUC_fold = train_image(epochs=EPOCHS,
                                            model=model_resnet,
                                            learning_rate=LEARNING_RATE,
                                            train_dl=train_loaders[fold],
                                            val_dl=val_loaders[fold],
                                            min_epoch_train=MIN_EPOCH_TRAIN,
                                            patience=PATIENCE,
                                            epsilon=EPSILON,
                                            log_file=os.path.join(LOG_FILE_1, f'log_res34_aug_fold_{fold}.csv'),
                                            model_save_path=os.path.join(MODEL_SAVE_PATH_, f'model_resnet34_aug_fold_{fold}.pth'))
                val_pAUC.append(val_pAUC_fold)
                csv_writer.writerow([fold, os.path.basename(os.path.join(MODEL_SAVE_PATH_, f'model_resnet34_aug_fold_{fold}.pth')), val_pAUC_fold]) # Logging avg pauc for the model trained on each fold
        best_model_fold_index = val_pAUC.index(max(val_pAUC))
        
        file_paths = [os.path.join(LOG_FILE_1, f'log_res34_aug_fold_{i}.csv') for i in range(FOLDS)]
        plot_metrics_from_files(file_paths, save_path=METRICS_PLOT_SAVE_PATH)

        return best_model_fold_index, None, None, None
    
    
    elif TRAIN_METADATA and not TRAIN_IMAGE:
        df_train, new_num_cols, new_cat_cols = feature_engineering(annotations_df_trunc)
        num_cols = [
            'age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 
            'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
            'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 
            'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
            'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
            'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
            'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
            'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
            'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
        ] + new_num_cols
        cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple"] + new_cat_cols
        train_cols = num_cols + cat_cols
        category_encoder = OrdinalEncoder(
            categories='auto',
            dtype=int,
            handle_unknown='use_encoded_value',
            unknown_value=-2,
            encoded_missing_value=-1,
        )

        X_cat = category_encoder.fit_transform(df_train[cat_cols])
        for c, cat_col in enumerate(cat_cols):
            df_train[cat_col] = X_cat[:, c]
        
        splits = get_stratified_splits(df_train)

        val_scores = []
        meta_models = []
        for fold in range(FOLDS):
            train_df, val_df = splits[fold]
            model_metadata = lgb.LGBMRegressor(**lgb_params)
            model_metadata.fit(train_df[train_cols], train_df["target"])
            preds = model_metadata.predict(val_df[train_cols])
            score = pauc_above_tpr(val_df["target"], preds)
            val_scores.append(score)
            meta_models.append(model_metadata)
            print(f"\nFold: {fold+1} - Partial AUC Score: {score:.5f}")

        return None, meta_models, train_cols, cat_cols
    

    elif TRAIN_IMAGE and TRAIN_METADATA:
        # Image part
        train_loaders, val_loaders, _ = get_loader(
            test_hdf5_path=TEST_HDF5_PATH,
            train_labels_df=annotations_df_trunc,
            train_hdf5_path=TRAIN_HDF5_PATH
        )
        val_pAUC = []
        with open(LOG_FILE_2, 'w', newline="") as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(['Fold', "Model_Name", "avg_val_pAUC"])
            for fold in range(FOLDS):
                model_resnet = load_model(model_save_path=RESNET34_HAM10000_WEIGHTS_PYTORCH)
                print(f"---------------\nTraining for fold: {fold+1}:\n---------------")
                val_pAUC_fold = train_image(epochs=EPOCHS,
                                            model=model_resnet,
                                            learning_rate=LEARNING_RATE,
                                            train_dl=train_loaders[fold],
                                            val_dl=val_loaders[fold],
                                            min_epoch_train=MIN_EPOCH_TRAIN,
                                            patience=PATIENCE,
                                            epsilon=EPSILON,
                                            log_file=os.path.join(LOG_FILE_1, f'log_res34_aug_fold_{fold}.csv'),
                                            model_save_path=os.path.join(MODEL_SAVE_PATH_, f'model_resnet34_aug_fold_{fold}.pth'))
                val_pAUC.append(val_pAUC_fold)
                csv_writer.writerow([fold, os.path.basename(os.path.join(MODEL_SAVE_PATH_, f'model_resnet34_aug_fold_{fold}.pth')), val_pAUC_fold]) # Logging avg pauc for the model trained on each fold
        best_model_fold_index = val_pAUC.index(max(val_pAUC))
        
        file_paths = [os.path.join(LOG_FILE_1, f'log_res34_aug_fold_{i}.csv') for i in range(FOLDS)]
        plot_metrics_from_files(file_paths, save_path=METRICS_PLOT_SAVE_PATH)
        

        # Meta part
        df_train, new_num_cols, new_cat_cols = feature_engineering(annotations_df_trunc)
        num_cols = [
            'age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 
            'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
            'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 
            'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
            'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
            'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
            'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
            'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
            'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
        ] + new_num_cols
        cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple"] + new_cat_cols
        train_cols = num_cols + cat_cols
        category_encoder = OrdinalEncoder(
            categories='auto',
            dtype=int,
            handle_unknown='use_encoded_value',
            unknown_value=-2,
            encoded_missing_value=-1,
        )

        X_cat = category_encoder.fit_transform(df_train[cat_cols])
        for c, cat_col in enumerate(cat_cols):
            df_train[cat_col] = X_cat[:, c]
        
        splits = get_stratified_splits(df_train)

        val_scores = []
        meta_models = []
        for fold in range(FOLDS):
            train_df, val_df = splits[fold]
            model_metadata = lgb.LGBMRegressor(**lgb_params)
            model_metadata.fit(train_df[train_cols], train_df["target"])
            preds = model_metadata.predict(val_df[train_cols])
            score = pauc_above_tpr(val_df["target"], preds)
            val_scores.append(score)
            meta_models.append(model_metadata)
            print(f"\nFold: {fold+1} - Partial AUC Score: {score:.5f}")

        return best_model_fold_index, meta_models, train_cols, cat_cols


    else:
        print('Bruh')
        return None, None, None, None

## Main

In [ ]:
image_best_idx, meta_models, train_cols, cat_cols = lesgooo(TRAIN_IMAGE, TRAIN_METADATA)

In [ ]:
print(f"Best Image Model: /kaggle/working/model_resnet34_aug_fold_{image_best_idx}.pth")

In [ ]:
'''
Paths
'''
# Kaggle
TEST_HDF5_PATH = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
MODEL_SAVE_PATH = f"/kaggle/working/model_resnet34_aug_fold_{image_best_idx}.pth"
SUBMISSION_FILE_PATH = "/kaggle/working/submission_image.csv"

## Srijan
# TEST_HDF5_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Data\\test-image.hdf5"
# MODEL_SAVE_PATH = f"D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\model_resnet34_aug_fold_{IDX}.pth"
# SUBMISSION_FILE_PATH = "D:\\ISIC 2024 - Skin Cancer Detection with 3D-TBP\\Codebase\\Classification_v2\\submission.csv"

In [ ]:
def predict(TRAIN_IMAGE, TRAIN_METADATA):
    if TRAIN_IMAGE and not TRAIN_METADATA:
        mod = load_model(model_save_path=MODEL_SAVE_PATH)
        test_loader = get_loader(test_hdf5_path=TEST_HDF5_PATH)
        create_submission(image_model = mod,
                          test_loader = test_loader, 
                          submission_file_path=SUBMISSION_FILE_PATH, 
                          train_image = TRAIN_IMAGE, train_metadata = TRAIN_METADATA)
    elif TRAIN_METADATA and not TRAIN_IMAGE:
        create_submission(meta_models=meta_models,
                          test_meta=TEST_META,
                          train_cols=train_cols,
                          cat_cols = cat_cols,
                          submission_file_path=SUBMISSION_FILE_PATH, 
                          train_image = TRAIN_IMAGE, train_metadata = TRAIN_METADATA)
    elif TRAIN_IMAGE and TRAIN_METADATA:
        mod = load_model(model_save_path=MODEL_SAVE_PATH)
        test_loader = get_loader(test_hdf5_path=TEST_HDF5_PATH)
        create_submission(image_model = mod,
                          test_loader = test_loader, 
                          meta_models=meta_models,
                          test_meta=TEST_META,
                          train_cols=train_cols,
                          cat_cols = cat_cols,
                          submission_file_path=SUBMISSION_FILE_PATH, 
                          train_image = TRAIN_IMAGE, train_metadata = TRAIN_METADATA)
    else:
        print("Bruh")
        exit()

In [ ]:
predict()